![My Image](https://raw.githubusercontent.com/ralf-42/Image/main/genai-banner-2.jpg)

<p><font size="5" color='grey'> <b>
OutputParser
</b></font> </br></p>


---

In [ ]:
#@title
#@markdown   <p><font size="4" color='green'>  Colab-Umfeld</font> </br></p>
# Installierte Python Version
import sys
print(f"Python Version: ",sys.version)
# Installierte LangChain Bibliotheken
print()

print("Installierte LangChain Bibliotheken:")
!pip list | grep '^langchain'
# Unterdrückt die "DeprecationWarning" von LangChain für die Memory-Funktionden
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning, module="langsmith.client")
warnings.filterwarnings("ignore", category=DeprecationWarning, module="langchain")

In [ ]:
#@title
#@markdown   <p><font size="4" color='green'>  SetUp API-Keys (setup_api_keys)</font> </br></p>
def setup_api_keys():
    """Konfiguriert alle benötigten API-Keys aus Google Colab userdata"""
    from google.colab import userdata
    import os
    from os import environ

    # Dictionary der benötigten API-Keys
    keys = {
        'OPENAI_API_KEY': 'OPENAI_API_KEY',
        'HF_TOKEN': 'HF_TOKEN',
        # Weitere Keys bei Bedarf
    }

    # Keys in Umgebungsvariablen setzen
    for env_var, key_name in keys.items():
        environ[env_var] = userdata.get(key_name)

    return {k: environ[k] for k in keys.keys()}

# Verwendung
all_keys = setup_api_keys()
# Bei Bedarf einzelne Keys direkt zugreifen
# WEATHER_API_KEY = all_keys['WEATHER_API_KEY']

# 1 | OutputParser
---

LangChain stellt eine Vielzahl von Ausgabeparsern bereit, die speziell darauf ausgelegt sind, Informationen aus den Ergebnissen großer Sprachmodelle (LLMs) effizient zu extrahieren und zu strukturieren. Diese Parser sind essenzielle Bestandteile der LangChain-Architektur und häufig zentrale Elemente sogenannter LangChain-Ketten. Solche Ketten bestehen aus konfigurierbaren Abfolgen von Operationen, die Modellausgaben verarbeiten und für weiterführende Anwendungen nutzbar machen.


**Warum sind OutputParser so wichtig?**
+ LLMs geben standardmäßig unstrukturierte Texte zurück.
+ OutputParser sind nötig, um das LLM-Output strukturiert weiterzuverarbeiten.
+ Besonders bei komplexen Anwendungen (z. B. Ketten mit mehreren Modellen, Agenten oder RAG-Systemen) müssen die Antworten klar definiert sein.

**Übersicht LangChain-Parser:**

<br>

| **Name**                  | **Beschreibung**                                                                 | **Supports Streaming** | **Input-Typ**         | **Output-Typ**           |
|---------------------------|----------------------------------------------------------------------------------|-------------------------|-----------------------|--------------------------|
| **ListOutputParser**      | Wandelt eine durch Kommas getrennte Liste in eine Python-Liste um.               | Nein                    | `str`                | `list[str]`             |
| **DateTimeParser**        | Wandelt eine Datums- oder Zeitzeichenkette in ein Python-Datetime-Objekt um.     | Nein                    | `str`                | `datetime.datetime`     |
| **StructuredOutputParser**| Wandelt Text basierend auf einem Schema in ein Dictionary um.                    | Nein                    | `str`                | `Dict[str, str]`        |
| **JSON Parser**           | Gibt ein JSON-Objekt basierend auf einem Pydantic-Modell zurück.                 | Ja                      | `str`                | `Message` (JSON Objekt) |
| **XML Parser**            | Wandelt XML-Ausgaben in ein Dictionary von Tags um.                              | Ja                      | `str`                | `dict`                 |
| **CSV Parser**            | Gibt eine Liste von durch Kommas getrennten Werten zurück.                       | Ja                      | `str`                | `List[str]`            |
| **OutputFixingParser**    | Versucht, fehlerhafte Ausgaben zu korrigieren, indem es ein anderes LLM aufruft. | Ja                      | Wraps anderer Parser  | Abhängig vom Parser     |
| **RetryWithErrorParser**  | Wiederholt die Eingabe bei Fehlern und sendet ursprüngliche Anweisungen mit.     | Ja                      | Wraps anderer Parser  | Abhängig vom Parser     |
| **Pydantic Parser**       | Gibt Daten basierend auf einem benutzerdefinierten Pydantic-Modell zurück.       | Ja                      | `str`                | `pydantic.BaseModel`   |
| **YAML Parser**           | Wie der Pydantic Parser, aber verwendet YAML zur Kodierung.                      | Ja                      | `str`                | `pydantic.BaseModel`   |
| **PandasDataFrame Parser**| Gibt Daten zurück, die für Pandas DataFrames geeignet sind.                       | Ja                      | `str`                | `dict`                 |
| **Enum Parser**           | Wandelt die Antwort in einen der angegebenen Enum-Werte um.                     | Ja                      | `str`                | `Enum`                 |

<br>

Diese Parser ermöglichen es, die Ausgaben von Sprachmodellen (LLMs) in strukturierte und nutzbare Formate umzuwandeln, was besonders für Anwendungen mit spezifischen Anforderungen nützlich ist.


# 2 | Strukturierter Parser (einfach)
---


Der **Structured Output Parser** in LangChain ermöglicht die strukturierte Verarbeitung der Ausgaben großer Sprachmodelle (LLMs). Dies ist besonders hilfreich, wenn Informationen aus mehreren Feldern extrahiert und kategorisiert werden müssen. Durch die Segmentierung der Modellausgabe in definierte Bereiche wird die Interpretation und Handhabung der Daten vereinfacht.

Während der **Pydantic/JSON-Parser** eine leistungsfähigere Lösung für komplexe Datenstrukturen darstellt, eignet sich der **Structured Output Parser** besonders für Umgebungen mit begrenzten Rechenressourcen oder für Modelle mit geringer Leistungsfähigkeit. Er bietet eine einfache und effiziente Möglichkeit, die Ausgabe zu strukturieren, ohne das System zu stark zu belasten.

Der erste Schritt bei der Nutzung dieses Parsers besteht in der Erstellung eines **ResponseSchemas**, das definiert, welche Werte extrahiert werden sollen. Jeder Wert wird dabei detailliert beschrieben. Anschließend wird der **StructuredOutputParser** aus einer Liste dieser Schemata erstellt.

In [ ]:
# Abschnitt 0: Install
!uv pip install --system --prerelease allow -q langchain-core langchain_community langchain_openai gradio

In [ ]:
# Abschnitt 1: Importe
from IPython.display import display, Markdown

# Neue strukturierte Imports für aktuelle LangChain-Versionen
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [ ]:
# Abschnitt 2: Konstanten definieren
MODEL = "gpt-4o-mini"
TEMPERATURE = 0.0

QUESTION = "Was ist Machine Learning?"

<p><font color='blue' size="4">
Ohne Nachbearbeitung mit einem Parser
</font></p>

In [ ]:
llm = ChatOpenAI(model=MODEL, temperature=TEMPERATURE)

In [ ]:
response = llm.invoke(QUESTION)

In [ ]:
for r in response:
    print(r)

<p><font color='blue' size="4">
Mit Nachbearbeitung mit einem Parser
</font></p>

In [ ]:
chain = llm | StrOutputParser()
response = chain.invoke(QUESTION)

In [ ]:
response

# 3 | Strukturieter Parser (mittel)

<p><font color='blue' size="4">
Structured Output Parser (mittel)
</font></p>

**Schritt 1:** Der **Parser** definiert ein Schema für die erwartete Ausgabe und verarbeitet die Antwort des LLMs in ein strukturiertes Format (z.B. ein Python-Dictionary).

In [ ]:
# 1. Parser-Schema definieren
RESPONSE_SCHEMAS = [
    ResponseSchema(name="question", description="Frage des Benutzers"),
    ResponseSchema(name="answer", description="Antwort auf die Frage des Benutzers"),
    ResponseSchema(name="source", description="Verwendete Quelle (Website) für die Antwort")
]

# 2. Parser erstellen
parser = StructuredOutputParser.from_response_schemas(RESPONSE_SCHEMAS)

# 3. Formatierungs Instruktion zu diesem Parser für das Prompt erstellen
FORMAT_INSTRUCTIONS = parser.get_format_instructions()

Wir können die einzelnen Elemente anzeigen, die wir zum Abrufen von Daten verwenden werden.

In [ ]:
print(RESPONSE_SCHEMAS)

In [ ]:
print(parser)

In [ ]:
print(FORMAT_INSTRUCTIONS)

**Schritt 2:** Verwendung des Schemas mit der Formatierungs Instruktion in einem Prompt.

In [ ]:
# 1. System-Prompt mit den Variablen (f-format) erstellen
SYSTEM_PROMPT = """
    Beantworte die folgende Frage so präzise wie möglich.
    Gib auch die Quelle deiner Information an.

    Frage: {question}

    {format_instructions}
"""

# 2. Prompt-Template generieren
prompt = PromptTemplate(
    template=SYSTEM_PROMPT,
    input_variables=["question"],
    partial_variables={"format_instructions": FORMAT_INSTRUCTIONS}
)

Wir können die Eingabeaufforderung anzeigen, die wir zum Abrufen von Daten verwenden werden.

In [ ]:
print(SYSTEM_PROMPT)

In [ ]:
print(prompt)

In [ ]:
# Funktion zur Aufbereitung der Druck-Ausgabe
def markdown_out(result):
    display(Markdown(f"**🧑 Mensch**"))
    display(Markdown(f"{result['question']}"))
    display(Markdown(f"**🤖 KI:**"))
    display(Markdown(f"{result['answer']}"))
    display(Markdown(f"**Quelle:** {result['source']}"))

<p><font color='black' size="5">
Einbindung des Parser in unterschiedliche Chain-Strukturen.
</font></p>

**Variante 0:** Ohne Chain-Struktur

In [ ]:
# LLM definieren - ohne weitere Kette
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)

# Aufruf
llm_response = llm.invoke(prompt.format(question=QUESTION))

# Antwort parsen
result = parser.parse(llm_response.content)

markdown_out(parsed_response)

**Variante 2:** Chain mit direkte Eingabe als Dictionary

In [ ]:
chain = (
    prompt
    | llm
    | parser
)

# Aufruf mit Dictionary
result = chain.invoke({"question": "Was ist Machine Learning?"})
markdown_out(result)

**Variante 3:** Chain mit Dictionary-Transformation `RunnablePassthrough`

In [ ]:
# Chain mit Dictionary-Transformation
from langchain_core.runnables import RunnablePassthrough

chain = (
    {"question": RunnablePassthrough()}  # Wandelt String in Dictionary um
    | prompt
    | llm
    | parser
)

# Aufruf mit einfachem String
result = chain.invoke(QUESTION)
markdown_out(result)

# 4 | Strukturierter Parser (komplex)
---

Nun wird ein Beispiel mit einer größeren Anzahl an Werten getestet. Das folgende Programm nimmt Texte in beliebigen Sprachen entgegen und übersetzt sie ins Englische, Spanische und Chinesische.

**Schritt 1:** Der **Parser** definiert ein Schema für die erwartete Ausgabe und verarbeitet die Antwort des LLMs in ein strukturiertes Format (z.B. ein Python-Dictionary).

In [ ]:
RESPONSE_SCHEMAS = [
    ResponseSchema(name="detected", description="Die Sprache ist Nutzer-Eingabe"),
    ResponseSchema(name="german", description="German translation"),
    ResponseSchema(name="english", description="English translation"),
    ResponseSchema(name="spanish", description="Spanish translation"),
    ResponseSchema(name="chinese", description="Chinese translation"),
]
parser = StructuredOutputParser.from_response_schemas(RESPONSE_SCHEMAS)

FORMAT_INSTRUCTIONS = parser.get_format_instructions()

**Schritt 2:** Verwendung des Schemas mit der Formatierungs Instruktion in einem Prompt.

In [ ]:
# 1. System-Prompt mit den Variablen (f-format) erstellen
SYSTEM_PROMPT = """
    Übersetze in die angegebenen Sprachen.\n{format_instructions}\n{question}
"""

# 2. Prompt-Template generieren
prompt = PromptTemplate(
    template=SYSTEM_PROMPT,
    input_variables=["question"],
    partial_variables={"format_instructions": FORMAT_INSTRUCTIONS}
)

**Variante 2:** Chain mit direkte Eingabe als *Dictionary*

Zunächst wird ein deutscher Satz getestet, um zu beobachten, wie die Übersetzung in die drei Zielsprachen erfolgt.

In [ ]:
chain = prompt | llm | parser
question = "Wann wurde Python eingeführt?"
chain.invoke({"question": question})

Dann wird ein französischer Satz getestet, um zu beobachten, wie die Übersetzung in die drei Zielsprachen erfolgt.

In [ ]:
chain = prompt | llm | parser
question = "Who rides so late through night and wind?"
chain.invoke({"question": question})

In [ ]:
result = chain.invoke({"question": question})

# 5 | Datenformat Parser
---

LangChain bietet eine breite Palette an Parsern, die unterschiedliche Datenformate verarbeiten können, was seine Einsatzmöglichkeiten erheblich erweitert. Es unterstützt unter anderem die nahtlose Integration von Pandas-Datenrahmen, kommagetrennten Listen, JSON-Strukturen sowie Datums- und Zeitobjekten. Diese Flexibilität ermöglicht eine effiziente Anpassung an verschiedene Arten von Dateneingaben und macht LangChain zu einem leistungsstarken Werkzeug für die Analyse und Verarbeitung von Daten. Im Folgenden werden einige dieser Parser genauer betrachtet, ihre praktischen Anwendungen demonstriert und aufgezeigt, wie sie zur Optimierung von Prozessen und zur Gewinnung wertvoller Erkenntnisse beitragen können.

<p><font color='black' size="5">
CSV
</font></p>

Der **CommaSeparatedListOutputParser** ermöglicht die Umwandlung von LLM-Ausgaben in eine durch Kommas getrennte Liste und extrahiert diese als native Python-Liste. Dies ist besonders nützlich, wenn strukturierte Daten aus einem Sprachmodell extrahiert und für weitere Verarbeitungsschritte verwendet werden sollen. Indem der Parser die Ausgabe direkt in eine Listenstruktur überführt, erleichtert er die Handhabung und Weiterverarbeitung der generierten Daten in verschiedenen Anwendungen.

In [ ]:
# Abschnitt 1: Importe
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# Abschnitt 2: Konstanten definieren
MODEL = "gpt-4o-mini"
TEMPERATURE = 0

# Abschnitt 3: Parser und Prompt
output_parser = CommaSeparatedListOutputParser()

format_instructions = output_parser.get_format_instructions()

chat_prompt = PromptTemplate(
    template="List ten {subject}.\n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions},
)

# Abschnitt 4: Chat-Komponenten initialisieren
chat_model = ChatOpenAI(
    model=MODEL,
    temperature=TEMPERATURE,
    n=1
)

# Abschnitt 5: Funktionen definieren
def generate_list(subject: str):
    """  Generiert eine Liste mit zehn Elementen zu einem gegebenen Thema.  """
    chain = chat_prompt | chat_model | output_parser
    return chain.invoke({"subject": subject})

# Abschnitt 6: Hauptprogramm
def main():
    """Startet die Hauptkonversation und gibt eine Liste von Städten aus."""
    cities = generate_list("Städte")
    print(type(cities), cities)

if __name__ == "__main__":
    main()

<p><font color='black' size="5">
JSON
</font></p>

Die Ausgabe des LLM kann in einem JSON-Format strukturiert werden.

In [ ]:
#
# Variante mit Prompt
#
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

prompt = (
    "Bitte gib mir eine Person mit Name und Alter im JSON-Format zurück, z.B. "
    '{"name": "Max", "age": 30}'
)

response = llm.invoke(prompt)
print("Antwort vom Modell:")
print(response)

In [ ]:
#
# Variante mit JsonOutputParser
#
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers.json import JsonOutputParser

# Einfaches JSON-Schema
schema = {
        "name": {"type": "string"},
        "age": {"type": "integer"}
}

# JsonOutputParser mit dem Schema erstellen
parser = JsonOutputParser(schema=schema)

# OpenAI LLM initialisieren
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

# Modell fragen, Antwort soll JSON sein
prompt = (
    "Please give as an answert name and age of a person in json-Format"
)

response = llm.invoke(prompt)
# print("Antwort vom Modell:")
# print(response)

# Antwort parsen
result = parser.invoke(response)
print("Geparstes Ergebnis:")
print(result)

In diesem Beispiel wird ein Satz erkannt, der als Englisch identifiziert wird, und anschließend ins Spanische, Französische und Chinesische übersetzt.

In [ ]:
# Abschnitt 1: Importe
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# Abschnitt 2: Funktionen
def translate_text(text: str) -> dict:
    """Übersetzt den Text in mehrere Sprachen"""
    # Parser ohne Pydantic-Modell konfigurieren
    parser = JsonOutputParser()
    llm = ChatOpenAI(model="gpt-4", temperature=0.0)

    # Prompt erstellen mit direkten Anweisungen zum JSON-Format
    prompt_template = """
    Übersetze die Benutzereingabe in mehrere Sprachen und gib das Ergebnis als JSON zurück.

    Dein JSON sollte folgende Felder enthalten:
    - detected: Die erkannte Ausgangssprache
    - german: Die deutsche Übersetzung
    - spanish: Die spanische Übersetzung
    - french: Die französische Übersetzung
    - chinese: Die chinesische Übersetzung

    Benutzereingabe: {input}
    """

    prompt = PromptTemplate(
        template=prompt_template,
        input_variables=["input"]
    )

    # Kette ausführen
    chain = prompt | llm | parser
    return chain.invoke({"input": text})

# Abschnitt 3: Hauptprogramm
# Beispieltext übersetzen und Ergebnisse ausgeben
result = translate_text("What is your name?")

print()
display(Markdown(f"**Erkannte Sprache:** {result['detected']}"))
display(Markdown(f"**Deutsch:** {result['german']}"))
display(Markdown(f"**Spanisch:** {result['spanish']}"))
display(Markdown(f"**Französisch:** {result['french']}"))
display(Markdown(f"**Chinesisch:** {result['chinese']}"))

In [ ]:
# Ausgabe json-Format
import json
print(json.dumps(result, indent=2, ensure_ascii=False))

<p><font color='black' size="5">
Datum/Uhrzeit
</font></p>

Langchain bietet mit dem **DatetimeOutputParser** eine spezielle Funktion zur Interpretation von Datums- und Zeitangaben aus Texten. Sie erkennt unterschiedliche Formate und wandelt diese in ein standardisiertes Zeitformat um. Dies ist besonders hilfreich für Anwendungen wie Terminplanung, Datenanalyse oder andere Bereiche, in denen eine präzise Verarbeitung zeitlicher Informationen erforderlich ist. Der **DatetimeOutputParser** erleichtert die Verarbeitung von Datums- und Uhrzeitangaben und trägt dazu bei, dass Anwendungen zeitbezogene Daten effizient verwalten und nutzen können.

In [ ]:
# Abschnitt 1: Importe
from langchain.output_parsers import DatetimeOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import OpenAI

# Abschnitt 2: Konstanten
TEMPLATE = """Beantworte die Frage zu genau wie möglich.
Wenn Du die Frage nach dem Datum nicht beantworten kannst, dann geben mir das Datum 01.01.1111.

{question}

{format_instructions}"""

# Abschnitt 3: Komponenten initialisieren
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.2,
    n=1
)

output_parser = DatetimeOutputParser()
prompt = PromptTemplate(
    template=TEMPLATE,
    input_variables=['question'],
    partial_variables={'format_instructions': output_parser.get_format_instructions()}
)

Wir können die Eingabeaufforderung anzeigen, die wir zum Abrufen von Daten verwenden werden.

In [ ]:
print(prompt)

Wir erstellen die Kette, die wir zum Parsen von Daten verwenden werden.

In [ ]:
chain = prompt | llm | output_parser

Wir werden nach zwei Daten fragen, einem realen und einem fiktiven.

In [ ]:
output = chain.invoke({"question": "Wann wurde die Progammiersprache Python eingeführt?"}) # 20.02.1991
print(output)

In [ ]:
output = chain.invoke({"question": "An welchem Tag startet der Krieg im Videospiel Fallout?"}) #  23.10.2077
print(output)


# 6 | Pydantic Parser
---

Pydantic ist eine Python-Bibliothek zur Datenvalidierung und Einstellungsverwaltung, die auf den Typanmerkungen von Python basiert. Sie ermöglicht eine effiziente und einfache Validierung sowie Umwandlung von Daten und nutzt dabei das standardisierte Typisierungssystem von Python.

**Wichtige Funktionen von Pydantic:**
- **Datenvalidierung:** Überprüft, ob Daten den erwarteten Formaten entsprechen, und konvertiert sie bei Bedarf in die korrekten Typen.
- **Unterstützung für Code-Editoren:** Dank der Nutzung von Python-Typanmerkungen bieten moderne Editoren Funktionen wie Autovervollständigung und Typprüfung für Pydantic-Modelle.
- **Fehlermeldungen:** Detaillierte und verständliche Fehlermeldungen helfen, Probleme bei der Datenvalidierung gezielt zu identifizieren.
- **Einstellungsverwaltung:** Erleichtert das Laden und Verwalten von Konfigurationsparametern aus verschiedenen Quellen, darunter Umgebungsvariablen und JSON-Dateien.
- **Erweiterbarkeit:** Modelle lassen sich durch Methoden und Eigenschaften anpassen, und benutzerdefinierte Validierungen können mithilfe von Pydantic-Dekoratoren hinzugefügt werden.
- **Integration mit anderen Bibliotheken:** Besonders in Verbindung mit FastAPI verbessert Pydantic die Verwaltung und Validierung von API-Daten erheblich.

Aufgrund seiner Flexibilität und Benutzerfreundlichkeit ist Pydantic besonders nützlich für Webentwicklung und datenintensive Anwendungen, bei denen eine zuverlässige Datenverarbeitung erforderlich ist.



**Pydantic in LangChain:**    
LangChain, eine Bibliothek zur Entwicklung von Anwendungen mit Sprachmodellen, nutzt Pydantic für die strukturierte Verarbeitung von Modellantworten. Der **PydanticOutputParser** gewährleistet, dass die Ausgaben eines Sprachmodells einer vorgegebenen Struktur entsprechen. Dies ist besonders vorteilhaft für Anwendungen, bei denen konsistente Datenformate entscheidend sind – beispielsweise bei der Datenextraktion, API-Antworten oder der automatisierten Weiterverarbeitung von Modell-Ausgaben.

Der Code nachfolgende demonstriert die Verwendung von Pydantic zur Validierung strukturierter Daten am Beispiel von Kochrezepten. Diese Implementierung zeigt, wie Datenstrukturen definiert, validiert und gegen spezifische Regeln geprüft werden können.

**Anwendungsszenario:**   
Stellen Sie sich vor, Sie entwickeln eine Koch-App oder eine Rezeptdatenbank, bei der die Qualität und Konsistenz der eingegebenen Rezepte entscheidend ist. Der Code bildet das Herzstück einer Validierungsschicht, die sicherstellt, dass alle Rezepte den definierten Standards entsprechen:

**Datenstruktur-Validierung:**
+ Jedes Rezept muss bestimmte Pflichtfelder enthalten (Name, Schwierigkeitsgrad, Zubereitungszeit, Zutaten).
+ Typprüfung: Die Felder müssen den korrekten Datentyp aufweisen (z.B. Zubereitungszeit als ganze Zahl).
+ Inhaltliche Validierung: Der Schwierigkeitsgrad darf nur bestimmte Werte annehmen, die Zubereitungszeit muss in einem realistischen Bereich liegen.


In [ ]:
# Import
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from typing import List

# 1. Datenstruktur definieren
class Rezept(BaseModel):
    name: str = Field(description="Name des Gerichts")
    schwierigkeitsgrad: str = Field(description="Schwierigkeitsgrad (leicht/mittel/schwer)")
    zubereitungszeit: int = Field(description="Zubereitungszeit in Minuten")
    zutaten: List[str] = Field(description="Liste der benötigten Zutaten")

# 2. Parser erstellen
parser = PydanticOutputParser(pydantic_object=Rezept)

# 3. LLM initialisieren
llm = ChatOpenAI(temperature=0.2)  # Niedrige Temperatur für konsistentere Ergebnisse

# 4. Prompt-Template erstellen
prompt_text = """
Erstelle ein strukturiertes Rezept für das folgende Gericht.

{format_instructions}

Gericht: {gericht}
"""

prompt = PromptTemplate(
    template=prompt_text,
    input_variables=["gericht"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

# 5. Funktion zum Abfragen des Rezepts
def rezept_abfragen(gericht_name):
    try:
        # Prompt formatieren
        formatted_prompt = prompt.format(gericht=gericht_name)

        # LLM-Antwort abrufen und parsen
        antwort = llm.invoke(formatted_prompt)
        rezept_objekt = parser.parse(antwort.content)
        return rezept_objekt
    except Exception as e:
        print(f"❌ Fehler bei der Rezeptabfrage: {e}")
        return None

# 6. Separate Funktion zur Überprüfung des Rezepts
def rezept_pruefen(rezept_objekt):
    try:
        if rezept_objekt is None:
            raise ValueError("Kein Rezeptobjekt vorhanden")

        # Überprüfung der Struktur und Datentypen
        assert isinstance(rezept_objekt.name, str), "Name ist kein String"
        assert isinstance(rezept_objekt.schwierigkeitsgrad, str), "Schwierigkeitsgrad ist kein String"
        assert isinstance(rezept_objekt.zubereitungszeit, int), "Zubereitungszeit ist keine Zahl"
        assert isinstance(rezept_objekt.zutaten, list), "Zutaten ist keine Liste"
        assert len(rezept_objekt.zutaten) > 0, "Zutaten-Liste ist leer"

        # Inhaltliche Überprüfung
        assert rezept_objekt.schwierigkeitsgrad.lower() in ["leicht", "mittel", "schwer"], \
            "Schwierigkeitsgrad muss 'leicht', 'mittel' oder 'schwer' sein"
        assert 5 <= rezept_objekt.zubereitungszeit <= 240, \
            "Zubereitungszeit sollte zwischen 5 und 240 Minuten liegen"

        print("✅ Überprüfung erfolgreich!")
        return True
    except Exception as e:
        print(f"❌ Fehler bei der Überprüfung: {e}")
        return False

In [ ]:
# 7. Normale Ausführung mit einem echten Rezept
def test_mit_echtem_rezept():
    print("=== Test mit echtem Rezept ===")
    gericht_name = "Spaghetti Carbonara"
    rezept = rezept_abfragen(gericht_name)

    if rezept and rezept_pruefen(rezept):
        print(f"\nRezept: {rezept.name}")
        print(f"Schwierigkeitsgrad: {rezept.schwierigkeitsgrad}")
        print(f"Zubereitungszeit: {rezept.zubereitungszeit} Minuten")
        print("Zutaten:")
        for zutat in rezept.zutaten:
            print(f"- {zutat}")

In [ ]:
test_mit_echtem_rezept()

In [ ]:
# 8. Dummy-Case für Fehlererzeugung
def test_mit_fehler_rezept():
    print("\n=== Test mit Dummy-Fehler-Rezept ===")
    # Erstellen eines fehlerhaften Rezeptobjekts
    from pydantic import create_model

    # Fehlerhafte Zubereitungszeit (als String statt int)
    fehlerhaftes_rezept = Rezept(
        name="Fehlerrezept",
        schwierigkeitsgrad="unmöglich",  # Ungültiger Schwierigkeitsgrad
        zubereitungszeit=300,  # Zu lange Zubereitungszeit
        zutaten=[]  # Leere Zutatenliste
    )

    if rezept_pruefen(fehlerhaftes_rezept) == False:
        print("Der Fehlertest war erfolgreich: Die Validierung hat die Fehler erkannt!")

In [ ]:
test_mit_fehler_rezept()

# 7 | Eigene Parser
---

In bestimmten Szenarien kann es sinnvoll sein, einen benutzerdefinierten Parser zu erstellen, um die Modellausgabe eindeutig zu formatieren.  

Dafür bietet sich die Verwendung von **RunnableLambda** oder **RunnableGenerator** in LCEL an, was für viele Fälle ein guten Ansatz ist.  

<p><font color='black' size="5">
Anwendungsfall
</font></p>



Large Language Models (LLMs) wie GPT-4 können Text generieren, der Code und erklärende Beschreibungen nahtlos vermischt. Dies kann zwar für Lern- und Dokumentationszwecke unglaublich nützlich sein, kann aber eine Herausforderung darstellen, wenn aus solchen Ausgaben mit gemischtem Inhalt nur der Code extrahiert und ausgeführt werden muss. Um dies zu beheben, implementieren wir eine einfache Funktion, die nicht-Python-Codezeilen aus einer gegebenen Textzeichenfolge entfernt.

Bei diesem Ansatz werden reguläre Ausdrücke verwendet, um Zeilen zu identifizieren und beizubehalten, die der typischen Python-Syntax entsprechen, während Zeilen verworfen werden, die beschreibender Text zu sein scheinen. Aufgrund der inhärenten Komplexität und Variabilität sowohl von Python-Code als auch von natürlicher Sprache kann diese Methode jedoch nie perfekt sein. Sie basiert auf heuristischen Mustern, die Code manchmal fälschlicherweise als Text klassifizieren oder umgekehrt.

Im nächsten Abschnitt werden wir untersuchen, wie ein anderes LLM beim Entfernen von Nicht-Python-Code helfen kann und möglicherweise eine ausgefeiltere und genauere Lösung bietet. Das folgende Beispiel enthält eine Mischung aus LLM-Kommentaren und generiertem Code.









In [ ]:
from langchain_core.runnables import RunnableLambda
from langchain_openai import ChatOpenAI
from IPython.display import display, Markdown


# 1. LLM-Modell definieren
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.9)

# 2. Funktion zur Trennung von Erläuterung und Code
def parse_ai_message(ai_message):
    """Trennt die Erläuterung und den Code aus einer AIMessage und gibt sie separat zurück."""
    text = ai_message.content  # Extrahiere den reinen Textinhalt der AIMessage

    if "```" in text:
        # Trennen der Erläuterung und des Codes
        parts = text.split("```")
        explanation = parts[0].strip()
        code = parts[1].strip() if len(parts) > 1 else ""
    else:
        # Falls kein Codeblock vorhanden ist, geben wir nur die Erläuterung zurück
        explanation = text.strip()
        code = ""

    return explanation, code  # Rückgabe von zwei separaten Werten

# 3. RunnableLambda für das Parsing erstellen
parser = RunnableLambda(parse_ai_message)

# 4. LLM und Parser verketten
chain = llm | parser

# 5. Eingabe an die Pipeline senden und Ergebnis ausgeben
prompt = """
    Erkläre mir, wie man eine einfache Funktion in Python erstellt und gib ein Beispiel.
    Verwende bei Formeln das Format $ Formel $
"""
explanation, code = chain.invoke(prompt)

display(Markdown("## Erläuterung"))
display(Markdown(explanation))
display(Markdown("## Code"))
print(code)

# A | Aufgabe
---

Die Aufgabestellungen unten bieten Anregungen, Sie können aber auch gerne eine andere Herausforderung angehen.


<p><font color='black' size="5">
JSON-Parser mit LangChain
</font></p>


**Ziel:** Verständnis für den Einsatz von `JsonOutputParser` in LangChain.

**Aufgabe:**  
1. Nutze den `JsonOutputParser` von LangChain, um eine KI-Antwort in JSON zu formatieren.  
2. Lasse ein Language Model (z. B. OpenAI GPT) eine Liste von drei zufälligen Städten in Deutschland generieren.  
3. Verwende den Parser, um die Ausgabe in ein JSON-Format umzuwandeln.

**Erwartete Ausgabe (Beispiel):**
```json
{
  "cities": ["Berlin", "Hamburg", "München"]
}
```



<p><font color='black' size="5">
Extraktion von Schlüsselwerten
</font></p>


**Ziel:** Nutzung des `PydanticOutputParser`, um strukturierte Daten aus natürlicher Sprache zu extrahieren.

**Aufgabe:**  
1. Definiere eine `Pydantic`-Datenklasse mit den Feldern: `name` (str), `alter` (int), `stadt` (str).  
2. Verwende ein Language Model, um aus einer gegebenen Beschreibung eine Person zu extrahieren.  
3. Parse die Antwort mit dem `PydanticOutputParser`.

**Beispiel-Eingabe:**
> "Max Mustermann ist 35 Jahre alt und lebt in Berlin."

**Erwartete Ausgabe (Pydantic-Modell):**
```json
{
  "name": "Max Mustermann",
  "alter": 35,
  "stadt": "Berlin"
}
```


<p><font color='black' size="5">
Parser für Listenformate
</font></p>


**Ziel:** Implementierung eines eigenen Parsers zur Umwandlung von KI-Ausgaben in Listen.

**Aufgabe:**  
1. Erstelle eine eigene Parser-Klasse, die eine durch Kommas getrennte Liste in ein Listenformat umwandelt.  
2. Verwende diesen Parser, um eine Liste von fünf beliebten Büchern aus einer Language-Model-Antwort zu extrahieren.

**Beispiel-Eingabe:**
> "Die Verwandlung, Faust, Der Prozess, Die Blechtrommel, Der Vorleser"

**Erwartete Ausgabe:**
```python
["Die Verwandlung", "Faust", "Der Prozess", "Die Blechtrommel", "Der Vorleser"]
```

<p><font color='black' size="5">
Kombination Parser & PromptTemplate
</font></p>


**Ziel:** Verwendung von `StructuredOutputParser` in Kombination mit `PromptTemplate`.

**Aufgabe:**  
1. Erstelle ein `PromptTemplate`, das eine strukturierte Antwort über ein Land ausgibt (Name, Hauptstadt, Einwohnerzahl).  
2. Verwende den `StructuredOutputParser`, um die Antwort in ein Dictionary zu konvertieren.

**Beispiel-Prompt:**
> "Gib mir Informationen zu Frankreich im folgenden JSON-Format: { 'name': '...', 'hauptstadt': '...', 'einwohner': ... }."

**Erwartete Ausgabe:**
```json
{
  "name": "Frankreich",
  "hauptstadt": "Paris",
  "einwohner": 67000000
}
```
